In [5]:
# Importing requird python packages
# Path to hsmpy package
HSMPY_PATH = r'\\CHCFPP01\Guest\MRajabi\Python Scripts'
import sys, os, arcpy, csv, json, math
sys.path.append(HSMPY_PATH)
import hsmpy3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import Image
from IPython.core.display import HTML 
import subprocess 
import webbrowser
import time
import copy
import scipy
from matplotlib.gridspec import GridSpec
from mailmerge import MailMerge
import shutil
import openpyxl
from time import gmtime, strftime
from ipywidgets import FloatProgress
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-12-10 16:08:34


In [1]:
import arcpy

In [7]:
Years = range(2005,2017)
Last5Y = range(12,17)
# Roadway and Crash Data
IRISPath = r'C:\Users\mr068144\Downloads\IRIS\IL_IRIS.gdb'
IRIS_Route = {y:os.path.join(IRISPath,'HWY'+str(y)+'_route') for y in Years}
IRIS_Table = {y:os.path.join(IRISPath,'HWY'+str(y)+'_table_HCurve') for y in Years}
CrashPath = r'C:\Users\mr068144\Downloads\IL Data\IL Crash Data\IL_Crash.gdb'
CrashData = {y:CrashPath + '\\CrashExtract_' + str(y) + '_GIS' for y in Years}

Project_Dir = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\ILSHSP_Tables'
OutputDir   =  Project_Dir + '\\WorkData\\State_Local_Urban_Rural'
DataDir = Project_Dir + '\\SourceData'

In [3]:
def StateLocal(PGN):
    try:
        PGN = str(PGN)
        return({True:'State',False:'Local'}[PGN[0] == 'S'])
    except:
        return('')
def RuralUrban(PGT):
    try:
        PGT = str(PGT)
        return({True:'Urban',False:'Rural'}[PGT.lower().find('urban')!=-1])
    except:
        return('')
def TraffwayConv(TW1,TW2):
    ConvDict = {
        0:'State Rural',
        1:'State Rural',
        2:'State Rural',
        3:'Local Rural',
        4:'State Rural',
        5:'State Urban',
        6:'State Urban',
        8:'Local Urban',
        9:'State Urban'
    }
    tw = ''
    if not math.isnan(TW1):
        tw = int(TW1)
    if not math.isnan(TW2):
        tw = int(TW2)
    if tw in ConvDict.keys():
        return(ConvDict[tw])
def GetCrashTime(y,m,d,s):
    y += 2000
    h = int(s.split(':')[0])
    mm = int(s.split(':')[1].split(' ')[0])
    ampm = s.split(':')[1].split(' ')[1]
    if ampm == 'PM':
            if h<12:
                h+=12
    return(datetime(int(y),int(m),int(d),int(h),int(mm)))
def ConvertLightCondition(l):
    if l in ['Darkness / Lighted Road','Darkness/ Lighted Road']:
        l = 'Darkness, Lighted Road'
    return(l)

In [4]:
#Crash_DF = pd.read_excel(Project_Dir + '\\PreviousFiles\\DataSheets_Crash12to16_KA_v2.xlsx',sheetname='12-16_Crash_KA')
#Crash_DF['KA'] = Crash_DF.Total_killed + Crash_DF.A_injuries
#print(Crash_DF.shape)
Crash_DF = pd.read_excel(Project_Dir + '\\SourceData\\CrashExtract_07to16_KA.xlsx')
Crash_DF['KA'] = Crash_DF.Total_killed + Crash_DF.A_injuries
Crash_DF['Date'] = [GetCrashTime(y,m,d,t) for y,m,d,t in zip(Crash_DF.Crash_Year,Crash_DF.Crash_Month,Crash_DF.Crash_Day,Crash_DF.Time_of_crash)]
Crash_DF = Crash_DF.sort_values('Date')
Crash_DF['LightCondition'] = [ConvertLightCondition(l) for l in Crash_DF.Light_condition]

print(Crash_DF.shape)
Occ_DF = pd.read_excel(Project_Dir + '\\SourceData\\PersonData_07to16_KA.xlsx')
print(Occ_DF.shape)
Veh_DF = pd.read_excel(Project_Dir + '\\SourceData\\VehicleExtract_07to16_KA.xlsx')
Veh_DF.index = Veh_DF.Case_number_CH2M
print(Veh_DF.shape)

EADF = pd.read_excel(Project_Dir + '\\SourceData\EA_Definition.xlsx',index_col=0)
EADF = EADF.sort_index()
EADF['Order'] = [1,4,5,3,2,7,13,6,8,9,10,11,12,14]
EADF = EADF.sort_values('Order')
EADF['Priority'] = [1,1,1,1,2,2,2,2,2,2,3,3,3,3]
EADF = EADF.sort_index()

Crash_DF.index = Crash_DF.CaseID_CH2M
Occ_DF.index   = Occ_DF.CaseNum_CH2M
Occ_DF['Pedestrian'] = Occ_DF.Pedestrain
Occ_DF['Crash_Year'] = Occ_DF.Year-2000
Occ_DF['A_injuries'] = [{True:1,False:0}[v==3] for v in Occ_DF['Injury Severity']]
Occ_DF['Total_killed'] = [{True:1,False:0}[v==4] for v in Occ_DF['Injury Severity']]
Occ_DF['KA'] = Occ_DF['Total_killed'] + Occ_DF['A_injuries']
Occ_DF['State_Local_Urban_Rural'] = Crash_DF.State_Local_Urban_Rural.loc[Occ_DF.index]
Occ_DF['District'] = Crash_DF.District.loc[Occ_DF.index]
Occ_DF['Date'] = Crash_DF.Date.loc[Occ_DF.index]
Occ_DF['UnrestrainedOccupants_ISHSP'] = Occ_DF['Unrestrained_Occupants_ISHSP']
for i,r in EADF[EADF.Acronym.isin(['RD','IN','SA','DF','HV','WZ','TN','IM','OD','YD','MC'])].iterrows():
    Occ_DF[r.CrashDataField] = Crash_DF[r.CrashDataField].loc[Occ_DF.index]

(108250, 111)
(137747, 56)
(177145, 58)


In [12]:
EADF = pd.read_excel(Project_Dir + '\\SourceData\EA_Definition.xlsx',index_col=0)
EADF = EADF.sort_index()
EADF['Order'] = [1,4,5,3,2,7,13,6,8,9,10,11,12,14]
EADF = EADF.sort_values('Order')
EADF['Priority'] = [1,1,1,1,2,2,2,2,2,2,3,3,3,3]
EADF = EADF.sort_index()
print(EADF.sort_values('Order')['Acronym'].tolist())

['RD', 'IM', 'UO', 'IN', 'SA', 'OD', 'YD', 'MC', 'HV', 'PD', 'PC', 'WZ', 'DF', 'TN']


In [5]:
EADF = pd.read_excel(Project_Dir + '\\SourceData\EA_Definition.xlsx',index_col=0)
EADF = EADF.sort_index()
EADF['Order'] = [1,4,5,3,2,7,13,6,8,9,10,11,12,14]
EADF = EADF.sort_values('Order')
EADF['Priority'] = [1,1,1,1,2,2,2,2,2,2,3,3,3,3]
EADF = EADF.sort_index()

In [6]:
FC_Dict = {
    'Interstate (PAS)':'01.Interstate',
        'Freeway and Expressway (Urban Only) (PAS)': '01.Interstate',
        'Other Principal Arterial (PAS)': '03.Principal Arterial – Other ',
        'Minor Arterial (Urban)': '04.Minor Arterial',
        'Minor Arterial (Non-Urban)': '04.Minor Arterial',
        'Major Collector (Non-Urban)': '05.Major Collector',
        'Collector (Urban)': '06.Minor Collector',
        'Minor Collector (Non-Urban)': '06.Minor Collector',
        'Local Road or Street (Urban)': '07.Local',
        'Local Road or Street (Non-Urban)': '07.Local',
        '(UNK)': '10.Unknown'}
Crash_DF['FunctionalClass'] = [FC_Dict[s] for s in Crash_DF.Roadway_functional_class_description]
Crash_DF['FC'] = [FC_Dict[s] for s in Crash_DF.Roadway_functional_class_description]
Crash_DF['Time'] = [datetime.time(datetime(2000,1,1,d.hour,0)).strftime('%I:%M %p') for d in Crash_DF.Date]
TimeOrder = [datetime.time(datetime(2000,1,1,d,0)).strftime('%I:%M %p') for d in range(0,24)]
Crash_DF['DayName'] = [d.weekday_name for d in Crash_DF.Date]
DayOrder = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
Crash_DF['Month']  = [d.strftime('%b') for d in Crash_DF.Date]
MonthOrder = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
CityClass = {0:'Unincorporated',
3:'Chicago',
4:'Pop. Under 2500',
5:'Pop. 2,500 - 5,000',
6:'Pop. 5,000 – 10,000',
7:'Pop. 10,000 – 25,000',
8:'Pop. 25,000 – 50,000',
9:'Pop. Over 50,000'}
Crash_DF['City_class'] = [CityClass[c] for c in Crash_DF.City_Class_Code]

In [7]:
for f in ['Type_of_crash','Time','DayName','LightCondition','Month','FunctionalClass','Weather','Roadway_surface','Cause1','County_name']:
    Occ_DF[f] = Crash_DF[f].loc[Occ_DF.index]

In [ ]:
hsmpy3.il.SHSP_NationalTrend_All(range(1994,2017),'10.National_trend1.png')
hsmpy3.il.SHSP_NationalTrend_All(range(2007,2017),'10.National_trend2.png')
hsmpy3.il.SHSP_EATrend(Crash_DF,EADF,'12.{}_trend.png'.format)
hsmpy3.il.SHSP_EAJAccard(Occ_DF[Occ_DF.Crash_Year.isin(Last5Y)],EADF,'61.{}_Associations.png'.format)
hsmpy3.il.SHSP_EABarStateLine(Occ_DF[Occ_DF.Crash_Year.isin(Last5Y)],'Type_of_crash',EADF,'34.{}_CrashType.png'.format)
hsmpy3.il.SHSP_EABarStateLine(Occ_DF[Occ_DF.Crash_Year.isin(Last5Y)],'Time',EADF,'35.{}_CrashTime.png'.format,TimeOrder)
hsmpy3.il.SHSP_EABarStateLine(Occ_DF[Occ_DF.Crash_Year.isin(Last5Y)],'DayName',EADF,'36.{}_CrashDay.png'.format,DayOrder)
hsmpy3.il.SHSP_EAStackStateLine(Occ_DF[Occ_DF.Crash_Year.isin(Last5Y)],'Age',EADF,'37.{}_Age.png'.format)
hsmpy3.il.SHSP_EABarStateLine(Occ_DF,'Age',EADF,'371.{}_Age.png'.format)
hsmpy3.il.SHSP_EAOverlap(Occ_DF[Occ_DF.Crash_Year.isin(Last5Y)],EADF,'38.{}_Overlap.png'.format)
hsmpy3.il.SHSP_EABarStateLine(Occ_DF[Occ_DF.Crash_Year.isin(Last5Y)],'LightCondition',EADF,'39.{}_LightCondition.png'.format)
hsmpy3.il.SHSP_EABarStateLine(Occ_DF[Occ_DF.Crash_Year.isin(Last5Y)],'Month',EADF,'41.{}_Month.png'.format,MonthOrder)
hsmpy3.il.SHSP_County(Occ_DF[Occ_DF.Crash_Year.isin(Last5Y)],EADF,'46.{}_County.png'.format)
hsmpy3.il.SHSP_EABarStateLine(Occ_DF[Occ_DF.Crash_Year.isin(Last5Y)],'FunctionalClass',EADF,'44.{}_FC.png'.format)
hsmpy3.il.SHSP_EABarStateLine(Occ_DF[Occ_DF.Crash_Year.isin(Last5Y)],'Weather',EADF,'45.{}_Weather.png'.format)
hsmpy3.il.SHSP_EABarStateLine(Occ_DF[Occ_DF.Crash_Year.isin(Last5Y)],'Roadway_surface',EADF,'47.{}_SurfaceCondition.png'.format)
hsmpy3.il.SHSP_EABarStateLine(Occ_DF[Occ_DF.Crash_Year.isin(Last5Y)],'Cause1',EADF,'48.{}_Cause.png'.format,Length=10)
#hsmpy3.il.Year_Percentage_OnePlot(Crash_DF,EADF,'Type_of_crash',[],'52.{}_TemporalType.png'.format)
#hsmpy3.il.Year_Percentage_Stacked(Crash_DF,EADF,'Type_of_crash',[],'53.{}_StackedType.png'.format)
#hsmpy3.il.Year_Percentage_SubPlot(Crash_DF,EADF,'Type_of_crash',[],'54.{}_SubPlotType.png'.format)
#hsmpy3.il.Year_Percentage_OnePlot(Crash_DF,EADF,'FC',[],'55.{}_TemporalFC.png'.format)
hsmpy3.il.Year_SLRU_Trend(Occ_DF,EADF,'56.{}_SLUR_Trends.png'.format)
hsmpy3.il.Year_SLRU_EMA(Occ_DF,EADF,'57.{}_SLUR_EMA.png'.format)
hsmpy3.il.SHSP_TimeTrend_Contour(Occ_DF[Occ_DF.Crash_Year.isin(Last5Y)],EADF,'66.{}_TimeTrendContour.png'.format)

In [13]:
SHSP_NationalTrend_All(range(1994,2017),'10.National_trend1.png')
SHSP_NationalTrend_All(range(2007,2017),'10.National_trend2.png')
SHSP_StateTrend_All(Crash_DF,'11.Statewide_trend.png')

In [11]:
def FindRatio(Y,YT):
    def objfun(x):
        return(sum([abs(y-float(yt)/x) for y,yt in zip(Y,YT)]))
    res = scipy.optimize.minimize(objfun, np.mean(YT)/np.mean(Y), method='nelder-mead',options={'xtol': 1e-8, 'disp': False})
    x = res.x[0]
    #ratio = np.arange(x*0.5,x*1.5,x*0.05)
    #plt.plot(ratio,[objfun(r) for r in ratio])
    #plt.vlines(x,plt.ylim()[0],plt.ylim()[1])
    #plt.show()
    return(x)

def SHSP_NationalTrend_All(Years,png_out):
    y = [2016 ,2015 ,2014 ,2013 ,2012 ,2011 ,2010 ,2009 ,2008 ,2007 ,2006 ,2005 ,2004 ,2003 ,2002 ,2001 ,2000 ,1999 ,1998 ,1997 ,1996 ,1995 ,1994 ]
    k = [34439,32539,30056,30202,31006,29867,30296,30862,34172,37435,38648,39252,38444,38477,38491,37862,37526,37140,37107,37324,37494,37241,36254]
    k = [37461,35485,32744,32893,33782,32479,32999,33883,37423,41259,42708,43510,42836,42884,43005,42196,41945,41717,41501,42013,42065,41817,40716]
    FARS_DF = pd.DataFrame()
    FARS_DF['KCrashes'] = k
    FARS_DF.index = y
    FARS_DF = FARS_DF.sort_index()
    FARS_DF = FARS_DF.loc[Years]
    X = list(FARS_DF.index)
    Y = list(FARS_DF.KCrashes)
    plt.figure(figsize=(8, 4), dpi=300, facecolor='w', edgecolor='k')
    plt.plot(X,Y,'-o')
    #plt.xlim(2006,2017)
    plt.xticks(X,[str(x) for x in X],rotation=90)
    plt.gca().set_yticklabels(['{:,.0f}'.format(y) for y in plt.gca().get_yticks()])
    plt.ylabel('Fatalities')
    plt.title('National - All Time')
    plt.grid()
    plt.savefig(png_out,transparent=True,dpi=1200)
    plt.close()
def SHSP_StateTrend_All(Crash_DF,png_out):
    y = [2016 ,2015 ,2014 ,2013 ,2012 ,2011 ,2010 ,2009 ,2008 ,2007 ,2006 ,2005 ,2004 ,2003 ,2002 ,2001 ,2000 ,1999 ,1998 ,1997 ,1996 ,1995 ,1994 ]
    k = [34439,32539,30056,30202,31006,29867,30296,30862,34172,37435,38648,39252,38444,38477,38491,37862,37526,37140,37107,37324,37494,37241,36254]
    k = [37461,35485,32744,32893,33782,32479,32999,33883,37423,41259,42708,43510,42836,42884,43005,42196,41945,41717,41501,42013,42065,41817,40716]

    FARS_DF = pd.DataFrame()
    FARS_DF['KCrashes'] = k
    FARS_DF.index = y
    FARS_DF = FARS_DF.sort_index()
    FARS_DF = FARS_DF.loc[Years]
    X = list(set(Crash_DF.Crash_Year))
    X = [x+2000 for x in X]
    X.sort()
    FARS_DF = FARS_DF.loc[X]
    Y = [Crash_DF[Crash_DF.Crash_Year==x-2000]['Total_killed'].sum() for x in X]
    YT = list(FARS_DF.KCrashes)
    r = FindRatio(Y,YT)
    YT = [float(yt)/r for yt in YT]
    plt.figure(figsize=(6, 4), dpi=300, facecolor='w', edgecolor='k')
    p1 = plt.plot(X,Y,'-o',label='State')
    p2 = plt.plot(X,YT,'--x',label ='Normalized National ({:0.2f}%)'.format(100.0/r))
    plt.xlim(X[0]-1,X[1]+1)
    plt.xticks(X,[str(x) for x in X],rotation=0)
    plt.gca().set_yticklabels(['{:,.0f}'.format(y) for y in plt.gca().get_yticks()])
    plt.ylabel('Fatalities')
    plt.title('Statewide')
    plt.grid()
    plt.plot([], [], ' ', label='Correlation: {:0.2f}'.format(np.corrcoef(x=Y,y=YT)[0][1]))
    plt.legend(loc='upper right',fancybox=True,framealpha=0.5, prop={'size': 9})
    plt.tight_layout()
    plt.savefig(png_out,transparent=True,dpi=1200)
    plt.close()


In [10]:
def Year_SLRU_EMA(Crash_DF,EADF,PNGName):
    import warnings
    warnings.filterwarnings('ignore')
    from scipy.optimize import curve_fit
    import matplotlib
    def LineFun(x, A, B):
            return A*x + B
    def EMA(Data,window):
        from scipy.interpolate import spline
        Data =Data.fillna(0)
        ema = pd.ewma(Data,span = window)[window-1:]
        #z = np.polyfit(ema.index, ema, 4)
        #f = np.poly1d(z)
        x = np.linspace(list(ema.index)[0], list(ema.index)[-1],100)
        try:
            y = spline(list(ema.index),ema,x)
            return(pd.Series(index=x,data=y))
        except:
            print(ema)
            return(pd.Series(index=x,data=0))
        #y = f(x)
    def Get_NM_EMA_Diff(Crash_EA_DF,SLUR):
                    cr = Crash_EA_DF[(Crash_EA_DF.State_Local_Urban_Rural==SLUR)].groupby(['Crash_Year'])['KA'].aggregate(sum)
                    cr.index = [2000+k for k in cr.index]
                    cr = cr.loc[range(2007,2017)]
                    cr = cr.fillna(0)
                    ema1 = pd.ewma(cr,span = 6)
                    ema2 = pd.ewma(cr,span = 3)
                    diff = ema2-ema1
                    diff = pd.Series([a/b for a,b in zip(diff.loc[diff.index[1:]],cr.loc[diff.index[:-1]])],index=range(2008,2017))
                    diff = diff.loc[range(2012,2017)]
                    return(diff)   
    my_cmap = matplotlib.cm.get_cmap('RdYlGn_r')
    my_norm = matplotlib.colors.Normalize(vmin=-1, vmax=1)
    barwidth = 0.5
    CL = pd.DataFrame(data=[[plt.cm.Set1(0.9/4*(i+1))] for i in range(4)],index=['State Urban','Local Urban','State Rural','Local Rural'])
    if 1<2:
            df = pd.DataFrame(Crash_DF.groupby(['Crash_Year','State_Local_Urban_Rural'])['KA'].aggregate(sum))
            df = df.unstack('State_Local_Urban_Rural')
            df.index = [2000+i for i in df.index]
            df.columns = [c[1] for c in df.columns]
            df = df.fillna(0)
            gs = matplotlib.gridspec.GridSpec(10, 1)
            n = len(df.columns)
            ic = 0
            M = list(matplotlib.markers.MarkerStyle.markers.keys())
            O = ['State Urban','Local Urban','State Rural','Local Rural']
            tPlot, axes = plt.subplots(nrows=4, ncols=1, sharex=True, sharey=False, gridspec_kw={'height_ratios':[3,1,3,1]})
            plt.figure(figsize=(7.5, 6.5), dpi=300, facecolor='w', edgecolor='k')
            for j,l in enumerate(['Urban','Rural']):
                plt.subplot(gs[5*j:5*j+3, :])
                for l2 in ['State','Local']:
                    ic+=0.9/4
                    plt.plot(df.index,df[' '.join([l2,l])],'-.o',label=' '.join([l2,l]), color=CL.loc[' '.join([l2,l]),0])
                    for k,win in enumerate([6,3]):
                        cr = Crash_DF[(Crash_DF.State_Local_Urban_Rural==' '.join([l2,l]))].groupby(['Crash_Year'])['KA'].aggregate(sum)
                        cr.index = [2000+k for k in cr.index]
                        cr = cr.fillna(0)
                        cr = cr.loc[range(2007,2017)]
                        #A,B = curve_fit(LineFun, cr.index, cr)[0]
                        plt.plot(EMA(cr,win),
                                 {0:'-',1:'--'}[k],
                                 color=CL.loc[' '.join([l2,l]),0],
                                 label=' - EMA {} Years'.format(win))
                plt.xlim(2006,2017)
                plt.xticks(range(2006,2017),[])
                yt = ['{:,.0f}'.format(x) for x in plt.gca().get_yticks()]
                yt[0] = ''
                yt[-1] = ''
                plt.gca().set_yticklabels(yt)
                plt.tick_params(axis='both', which='major', labelsize=6)
                plt.tick_params(axis='both', which='minor', labelsize=6)
                plt.legend(loc='upper left',fancybox=True,framealpha=0.5, prop={'size': 7},ncol=2)
                plt.grid()
                if j==0:
                    plt.title('Statewide')
                plt.subplot(gs[5*j+3, :])
                l2  = 'State'
                plt.gca().set_axis_bgcolor(CL.loc[' '.join([l2,l]),0])
                diff_S = Get_NM_EMA_Diff(Crash_DF[Crash_DF[r.CrashDataField]=='X'],' '.join(['State',l]))
                diff_L = Get_NM_EMA_Diff(Crash_DF[Crash_DF[r.CrashDataField]=='X'],' '.join(['Local',l]))
                L = [abs(v) for v in list(pd.concat([diff_S,diff_L]))]
                yl = [-max(L),max(L)]
                diff = diff_S
                plt.bar(diff.index,diff,align='center',color = my_cmap(np.sign(diff)),label='{} {}, EMA3-EMA6 Indicator'.format(l2,l),width=barwidth)
                plt.hlines(0,2006,2017,'black')
                plt.xlim(2006,2017)
                plt.ylim(yl[0],yl[1])
                plt.xticks(range(2006,2017),[])
                yt = ['{:,.1%}'.format(x) for x in plt.gca().get_yticks()]
                yt = [{True:y,False:''}[z in [1,len(yt)-2]] for z,y in enumerate(yt)]
                plt.gca().set_yticklabels(yt)
                plt.tick_params(axis='both', which='major', labelsize=6)
                plt.tick_params(axis='both', which='minor', labelsize=6)
                plt.grid()
                plt.legend(loc='upper left',fancybox=True,framealpha=0.5, prop={'size': 7},ncol=1)
                plt.subplot(gs[5*j+4, :])
                l2 = 'Local'
                plt.gca().set_axis_bgcolor(CL.loc[' '.join([l2,l]),0])
                diff = diff_L
                plt.bar(diff.index,diff,align='center',color=my_cmap(np.sign(diff)),label='{} {}, EMA3-EMA6 Indicator'.format(l2,l),width=barwidth)
                plt.hlines(0,2006,2017,'black')
                plt.xlim(2006,2017)
                plt.ylim(yl[0],yl[1])
                plt.xticks(range(2006,2017),[])
                yt = ['{:,.1%}'.format(x) for x in plt.gca().get_yticks()]
                yt = [{True:y,False:''}[z in [1,len(yt)-2]] for z,y in enumerate(yt)]
                plt.gca().set_yticklabels(yt)
                plt.tick_params(axis='both', which='major', labelsize=6)
                plt.tick_params(axis='both', which='minor', labelsize=6)
                plt.grid()
                plt.legend(loc='upper left',fancybox=True,framealpha=0.5, prop={'size': 7},ncol=1)
            plt.xticks(df.index,df.index,rotation=0)
            plt.tight_layout()
            plt.subplots_adjust(wspace=0, hspace=0)
            plt.savefig('SLUR_Statewide.png',transparent=False,DeprecationWarning=1200)
            plt.close()
            plt.close()
Year_SLRU_EMA(Occ_DF,EADF,'57.{}_SLUR_EMA.png'.format)

In [ ]:
SHSP_TimeTrend_Contour(Occ_DF[Occ_DF.Crash_Year.isin(Last5Y)],EADF,'66.{}_TimeTrendContour.png'.format)

In [39]:
hsmpy3.il.SHSP_County(Crash_DF[(Crash_DF.Crash_Year.isin(Last5Y)) & (Crash_DF.County_name!='Cook')],EADF[EADF.Name=='Pedestrian'],'62.{}_County.png'.format)

In [80]:
hsmpy3.il.SHSP_EABarStateLine(Occ_DF[Occ_DF.Crash_Year.isin(Last5Y)],'Cause1',EADF,'48.{}_Cause.png'.format,Length=10)

In [38]:
hsmpy3.il.SHSP_EABarStateLine(Occ_DF[Occ_DF.Crash_Year.isin(Last5Y)],'PED_Action',EADF[EADF.Acronym.isin(['PD','PC'])],'63.{}_PedAction.png'.format)
hsmpy3.il.SHSP_EABarStateLine(Occ_DF[Occ_DF.Crash_Year.isin(Last5Y)],'PED_Location',EADF[EADF.Acronym.isin(['PD','PC'])],'64.{}_PedLocation.png'.format)

In [9]:
#  All KAs by EA
EADF = EADF.sort_index()
EADF['Order'] = [1,4,5,3,2,7,13,6,8,9,10,11,12,14]
EADF = EADF.sort_values('Order')
EADF['Priority'] = [1,1,1,1,2,2,2,2,2,2,3,3,3,3]
EADF = EADF.sort_index()

df = pd.DataFrame(columns=['K_10_14','A_10_14','KA_10_14','K_12_16','A_12_16','KA_12_16'])
for i,r in EADF.iterrows():
    eadf = Occ_DF[(Occ_DF.Crash_Year.isin([10,11,12,13,14])) & (Occ_DF[r.CrashDataField]=='X')]
    eadf2 = Occ_DF[(Occ_DF.Crash_Year.isin([15,16,12,13,14])) & (Occ_DF[r.CrashDataField]=='X')]
    df.loc[r.Name] = [eadf.Total_killed.sum(),eadf.A_injuries.sum(),eadf.KA.sum(),eadf2.Total_killed.sum(),eadf2.A_injuries.sum(),eadf2.KA.sum()]
df.loc[EADF.sort_values('Order').Name]

,K_10_14,A_10_14,KA_10_14,K_12_16,A_12_16,KA_12_16
Roadway Departure,2483,19279,21762,2579,18626,21205
Impaired Driver,2088,8331,10419,2253,7900,10153
Unrestrained Occupants,1396,5288,6684,1375,4984,6359
Intersection Related,1178,26397,27575,1239,26078,27317
Speeding/Aggressive Driver,1108,12884,13992,1159,12982,14141
Older Driver,848,9593,10441,972,10000,10972
Young Driver,694,12240,12934,698,11789,12487
Motorcycle,703,5432,6135,735,5111,5846
Heavy Vehicle,672,4426,5098,697,4482,5179
Pedestrian,641,4525,5166,689,4594,5283


In [21]:
df = Occ_DF[(Occ_DF.Pedestrian=='X') & (Occ_DF.Year.isin(range(2010,2015)))]
df.groupby('LightFARS')['Total_killed'].aggregate(sum)

LightFARS
Dark         438
Dawn/Dusk     22
Daylight     180
Unknown        1
Name: Total_killed, dtype: int64

In [26]:
hsmpy3.il.SHSP_EABarStateLine(Occ_DF[(Occ_DF.Crash_Year.isin(range(7,17))) & (Occ_DF['Injury Severity']==3)],'LightFARS',EADF[EADF.Acronym=='PD'],'67.{}_A_LightFARS.png'.format)
hsmpy3.il.SHSP_EABarStateLine(Occ_DF[(Occ_DF.Crash_Year.isin(range(7,17))) & (Occ_DF['Injury Severity']==4)],'LightFARS',EADF[EADF.Acronym=='PD'],'67.{}_K_LightFARS.png'.format)
hsmpy3.il.SHSP_EABarStateLine(Occ_DF[(Occ_DF.Crash_Year.isin(range(7,17)))],'LightFARS',EADF[EADF.Acronym=='PD'],'67.{}_KA_LightFARS.png'.format)

In [24]:
Occ_DF['Injury Severity'].value_counts()


3    127753
4      9994
Name: Injury Severity, dtype: int64

In [20]:
Occ_DF['LightFARS'] = [{'Darkness':'Dark','Darkness, Lighted Road':'Dark','Dawn':'Dawn/Dusk','Daylight':'Daylight','Dusk':'Dawn/Dusk','Unknown':'Unknown'}[v] for v in  Occ_DF.LightCondition]

In [ ]:
Crash_DF['DayOfYear'] = [ for d in Crash_DF['DATE']]
Crash_DF['Day'] = Crash_DF.DATE.dt.strftime('%Y-%m-%d')
plt.figure(figsize=(8, 26), dpi=300, facecolor='w', edgecolor='k')
df = pd.DataFrame(Crash_DF.Day.value_counts().sort_index())
ylim = (min(df.Day),max(df.Day))
for i,year in enumerate(range(2004,2017)):
    plt.subplot(13,1,i+1)
    df = pd.DataFrame(Crash_DF[Crash_DF.YEAR==year].Day.value_counts().sort_index())
    df = df.fillna(0)
    plt.plot(list(df.Day))
    plt.xlim(1,366)
    plt.ylim(ylim[0],ylim[1])
    plt.grid()
    plt.title(year,x=0.5,y=.8)
plt.subplots_adjust(wspace=0, hspace=0)
plt.savefig('CrashOscilator.png',transparent=True,dpi=3000)
plt.show()